# Get RGTDB Entries for One User

https://rgtdb.com/user/72763

In [178]:
import pandas as pd
from datetime import datetime as dt
import datetime

dfs = pd.read_html('https://rgtdb.com/user/72763')
df = dfs[1]

## Convert date to datetime, attempt to guess the year

This is going to have troubles around New Year

In [179]:
event_year = str(dt.today().year)
year_subtract = 0
last_month = 13

for index, row in df.iterrows():
    event_month = int(row['Date'][:2])

    if event_month > last_month:
        year_subtract += 1
        event_year = str(dt.today().year - year_subtract)
        print(event_year + '-' + str(event_month))

    #print(event_year + '-' + row['Date'])

    df.loc[index, 'Year_Date'] = event_year + '-' + row['Date']

    last_month = event_month
    
df['Year_Date'] = pd.to_datetime(df['Year_Date'])
df.set_index('Year_Date', inplace=True)
df

2020-12


,Date,Name
Year_Date,,
2021-02-28 09:00:00,02-28 09:00,GFNS JENSIE E-FONDO
2021-02-21 15:00:00,02-21 15:00,Lou's Sunday Group Ride
2021-02-21 14:00:00,02-21 14:00,MRETT13 Lou's Team
2021-02-21 09:00:00,02-21 09:00,GFNS GOLDEN E-FONDO
2021-02-20 15:00:00,02-20 15:00,Lou's Saturday Group Ride
...,...,...
2020-11-19 04:45:00,11-19 04:45,Thursday Sweet Spot AM
2020-10-25 12:30:00,10-25 12:30,L’ENFER EN FRANCE
2020-10-24 15:00:00,10-24 15:00,Flemish For Beginners


## Personal Calendar for Signups

g2oh32rptulp8f17e3bdiq9iec@group.calendar.google.com

In [180]:
from gcsa.event import Event as gcEvent
from gcsa.google_calendar import GoogleCalendar

import socket
socket.setdefaulttimeout(300) # 5 minutes

EMAIL_FOR_CAL = 'g2oh32rptulp8f17e3bdiq9iec@group.calendar.google.com'

calendar = GoogleCalendar(EMAIL_FOR_CAL)


In [181]:
df.index.max() 


Timestamp('2021-02-28 09:00:00')

## Read Events in Calendar which Are Within Timeframe of RGDB Data

Add one day to timeframe to compensate for unpredictable time zone handling.

Note IDs of existing events so we don't re-create them.

In [182]:
df['cal_id'] = None
df['event_obj'] = None

for event in calendar.get_events(time_min=df.index.min() - datetime.timedelta(days=1), time_max=df.index.max() + datetime.timedelta(days=1), timezone='UTC'):
    if event.start in df.loc[df['Name'] == event.summary].index:
        df.at[event.start, 'cal_id'] = event.event_id
        df.at[event.start, 'event_obj'] = event
        print('found ', event, event.timezone, event.event_id)



found  2021-02-18 08:00:00+01:00 - Thursday under/overs UTC v3smr5ptv43kln460d8hl3lnu0
found  2021-02-17 06:00:00+01:00 - The Recovery Ride UTC e67fqdd5v9he3ufbsm66cken6s
found  2021-02-16 08:00:00+01:00 - Tuesday SS training UTC 4u2h376ijcko6kq7cqf55mm1bk
found  2021-02-14 16:00:00+01:00 - Lou's Sunday Group Ride UTC 2o41cs42lum3otkhogki59174k
found  2021-02-14 15:00:00+01:00 - MRETT12 Lou's Team UTC 3a4m18l8ormk5jf55nkvmj500o
found  2021-02-14 10:00:00+01:00 - GFNS HIGHLANDS E-FONDO UTC b71bff7rlakbpcj39a8fiaor6g
found  2021-02-13 16:00:00+01:00 - Lou's Saturday Group Ride UTC t3sf7719reugfrapovo8v1ed5c
found  2021-02-12 16:00:00+01:00 - Race to the Light House UTC 31ifkf67rec5e4p7tchik4omek
found  2021-02-11 08:00:00+01:00 - Thursday SST UTC hss4cil1fnvqijjdeoicb5phns
found  2021-02-10 08:00:00+01:00 - Breakfast Club UTC 20uvrehtik82pn449pidq6ii34
found  2021-02-09 16:00:00+01:00 - Gran Premio Pienza UTC oagd077g6ip32htaoru8aeipoo
found  2021-02-09 16:00:00+01:00 - Chat Laps UTC rgs

In [183]:
df

,Date,Name,cal_id,event_obj
Year_Date,,,,
2021-02-28 09:00:00,02-28 09:00,GFNS JENSIE E-FONDO,2qjhkp9cr1h1nn8e2d6j7ulf4k,2021-02-28 10:00:00+01:00 - GFNS JENSIE E-FONDO
2021-02-21 15:00:00,02-21 15:00,Lou's Sunday Group Ride,None,None
2021-02-21 14:00:00,02-21 14:00,MRETT13 Lou's Team,None,None
2021-02-21 09:00:00,02-21 09:00,GFNS GOLDEN E-FONDO,lef49el7mvujdbtk51l2qgb0mo,2021-02-21 10:00:00+01:00 - GFNS GOLDEN E-FONDO
2021-02-20 15:00:00,02-20 15:00,Lou's Saturday Group Ride,None,None
...,...,...,...,...
2020-11-19 04:45:00,11-19 04:45,Thursday Sweet Spot AM,3mduq5jttt435p8prt2lifl148,2020-11-19 05:45:00+01:00 - Thursday Sweet Spo...
2020-10-25 12:30:00,10-25 12:30,L’ENFER EN FRANCE,vjm85gl7mj4uk14pua7uooefls,2020-10-25 13:30:00+01:00 - L’ENFER EN FRANCE
2020-10-24 15:00:00,10-24 15:00,Flemish For Beginners,mj7c5ohbsvjf86j3gvq7gokauk,2020-10-24 17:00:00+02:00 - Flemish For Beginners


In [184]:
for index, row in df.iterrows():

    if row['cal_id'] == None:

        print('* Adding:', index, row['Name'])

        evntColor = '1'

        event = gcEvent(
            str(row['Name']),
            start=index,
            timezone='UTC',
            color = evntColor
        )

        #print('ID before add:', event.event_id)
        ret_event = calendar.add_event(event)
        print('ID after add:', event.event_id, 'Returned event ID:', ret_event.event_id)

    else: 
        print('Skipping', index, row['Name'])

Skipping 2021-02-28 09:00:00 GFNS JENSIE E-FONDO
* Adding: 2021-02-21 15:00:00 Lou's Sunday Group Ride
ID after add: None Returned event ID: nq3nfqhh2elek93mejmv70p9ek
* Adding: 2021-02-21 14:00:00 MRETT13 Lou's Team
ID after add: None Returned event ID: fi3r5o44hdj2sejsbncb860s50
Skipping 2021-02-21 09:00:00 GFNS GOLDEN E-FONDO
* Adding: 2021-02-20 15:00:00 Lou's Saturday Group Ride
ID after add: None Returned event ID: nbc2hqtst2u0h5ib6fh0gbs8b0
Skipping 2021-02-19 15:00:00 Lou's Fri Drafting Clinic
Skipping 2021-02-18 15:00:00 Lou's Thurs Draft Clinic
Skipping 2021-02-18 07:00:00 Thursday under/overs
Skipping 2021-02-17 05:00:00 The Recovery Ride
Skipping 2021-02-16 15:00:00 Lous Tues drafting clinic
Skipping 2021-02-16 07:00:00 Tuesday SS training
Skipping 2021-02-14 15:00:00 Lou's Sunday Group Ride
Skipping 2021-02-14 14:00:00 MRETT12 Lou's Team
Skipping 2021-02-14 09:00:00 GFNS HIGHLANDS E-FONDO
Skipping 2021-02-13 15:00:00 Lou's Saturday Group Ride
Skipping 2021-02-12 15:00:00 R

In [185]:
#event = df['event_obj'].iloc[4]
#event.id

In [186]:
#calendar.update_event(event)


In [187]:
#event.summary